In [1]:
from stable_baselines.common.bit_flipping_env import BitFlippingEnv

env = BitFlippingEnv(n_bits=10)

obs = env.reset()

for k, v in obs.items():
    print(f"{k}: {v}")


/home/krholshe/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/krholshe/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/krholshe/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/krholshe/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.

observation: [0 0 1 1 0 1 1 0 0 1]
achieved_goal: [0 0 1 1 0 1 1 0 0 1]
desired_goal: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [13]:
arr = np.arange(10)
' '.join(arr.astype('U'))

'0 1 2 3 4 5 6 7 8 9'

In [2]:
print(env.action_space)
a = env.action_space.sample()
a

Discrete(10)


9

In [2]:
class BitFlippingWrapper(gym.Wrapper):
    def __init__(self, env):
        super().__init__(env)
        self.n_bits = self.env.observation_space['observation'].n
        self.observation_space = gym.spaces.MultiBinary(2 * self.n_bits)
        self._a = None
        self._s = None
    
    @staticmethod
    def _concat(obs):
        return np.concatenate([obs['observation'], obs['desired_goal']])
    
    def reset(self):
        obs = self.env.reset()
        self._s = obs['observation']
        self._a = None
        return self._concat(obs)
        
    def step(self, a):
        obs, r, done, info = self.env.step(a)
        self._s = obs['observation']
        self._a = a
        return self._concat(obs), r, done, info
    
    def render(self, *args, **kwargs):
        if self._s is not None:
            s = list(map(str, self._s))
            if self._a is not None:
                s[self._a] = gym.utils.colorize(s[self._a], "yellow", highlight=True)
            print(''.join(s))


# env = BitFlippingEnv(n_bits=10)
# env = BitFlippingWrapper(env)
# env = km.wrappers.TrainMonitor(env, tensorboard_dir=tbdir)

In [3]:
shutil.rmtree('tensorboard/')

In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from collections import deque, namedtuple

%reload keras_gym.caching.experience_replay keras_gym.caching keras_gym
import keras_gym as km


K.clear_session()

# km.enable_logging()
tbdir = datetime.datetime.now().strftime('tensorboard/%H.%M.%S')
active = False

env = BitFlippingEnv(n_bits=10)
env = BitFlippingWrapper(env)
env = km.wrappers.TrainMonitor(env, tensorboard_dir=tbdir)

class Linear(km.FunctionApproximator):
    def body_q1(self, S, A):
        X = S
        X = keras.layers.Lambda(lambda x: K.cast(x, 'float32'))(X)
        if active:
            X = keras.layers.Lambda(lambda x: x[:, :env.n_bits])(X)
        X = keras.layers.Lambda(lambda args: tf.einsum('ij,ik->ijk', *args))([X, A])
        print(X)
        X = keras.layers.Flatten()(X)
        X = keras.layers.Dense(256, activation='relu')(X)
        return X


func = Linear(env, lr=0.01)
q = km.QTypeI(func, gamma=1, bootstrap_n=10, update_strategy='q_learning')
pi = km.EpsilonGreedy(q, epsilon=0.01)


episode_cache = []
buffer = km.caching.ExperienceReplayBuffer.from_value_function(
    value_function=q, batch_size=10, capacity=100000)


for ep in range(1000):
    s = env.reset()

    for t in range(env.max_steps):
        a = pi(s)
        s_next, r, done, info = env.step(a)

        # add transition to replay buffer and to episode cache
        r = (r + 1) / 2
        transition = (s, a, r, done, 2 * ep)
        buffer.add(*transition)
        episode_cache.append(transition)
        
        # update q-function (after experience warmup)
        if env.T > 100:
            q.batch_update(*buffer.sample())

        if done:
            j = np.random.randint(1, len(episode_cache) + 1)
            if j == len(episode_cache):
                # goal = terminal state
                g = s_next[:env.n_bits]
            else:
                # goal = would-be terminal state
                g = episode_cache[j][0][:env.n_bits]

            for i, (s, a, r, done, ep) in enumerate(episode_cache[:j]):
                
                # update goal (either active or passive transformation)
                if active:
                    s = np.concatenate([s[:env.n_bits], g])
                else:
                    s[:env.n_bits] = (s[:env.n_bits] + g + 1) % 2
                
                # update reward
                done = done or i == j - 1
                r = 1. if done else 0.
                
                # insert hindsight-updated transition into buffer
                buffer.add(s, a, r, done, ep + 1)
            
            episode_cache.clear()
            break

        s = s_next



module reloaded: keras_gym.caching.experience_replay
module reloaded: keras_gym.caching
module reloaded: keras_gym

Tensor("lambda_1/einsum/transpose_2:0", shape=(?, 20, 10), dtype=float32)
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [15]:
pi.epsilon = 0

s = env.reset()
env.render()
for t in range(env.max_steps):
    a = pi(s)
    s, _, done, _ = env.step(a)
    env.render()
    if done:
        break

0111000100
0111000101
1111000101
1111100101
1111101101
1111111101
1111111111
